# Import Packages

In [1]:
import json
import pandas as pd
import numpy as np
from sw_functions import sub_df, load_main_monsters, create_sw_adjacency_matrix

# Load Cards and Create Adjacency Matrix

In [ ]:
MAIN_MONSTERS = load_main_monsters()
SW_ADJACENCY_MATRIX = create_sw_adjacency_matrix(MAIN_MONSTERS) #small world adjacency matrix of all cards

# Find Best Bridges Function

In [ ]:
def find_best_bridges(deck_monster_names, required_target_names):
    #inputs: list of monster names and list of monsters that are required to connect with the small world bridges
    #output: The bridges that connect the most cards in your deck and connect with all the required targets
    deck_monster_names = list(set(deck_monster_names) | set(required_target_names)) #union names
    deck_indices = sub_df(MAIN_MONSTERS, deck_monster_names, 'name').index #indices of monsters in deck
    required_indices = sub_df(MAIN_MONSTERS, required_target_names, 'name').index #indices of required connections

    num_required_bridges = len(required_target_names) #number of cards required to connect with one bridge

    required_monster_matrix = SW_ADJACENCY_MATRIX[required_indices,:] #array corresponding to (required connection monsters) by (all monsters)
    num_bridges_to_required_cards = required_monster_matrix.sum(axis=0) #number of required connections satisfied by all monsters
    required_bridge_mask = num_bridges_to_required_cards==num_required_bridges
    bridges = MAIN_MONSTERS[required_bridge_mask] #data frame of monsters connecting all required targets
    required_bridge_indices = bridges.index #indices of monsters that satisfy all required connections
    if len(bridges)==0:
        print('There are no monsters that bridge all required targets.')
        return

    #subset of adjacency matrix corresponding to (deck monsters) by (monsters with connections to the required cards)
    bridge_matrix = SW_ADJACENCY_MATRIX[deck_indices,:][:,required_bridge_indices]

    num_deck_bridges = bridge_matrix.sum(axis=0)
    largest_num_bridges = max(num_deck_bridges)
    most_deck_bridges_mask = num_deck_bridges==largest_num_bridges
    most_deck_bridges = bridges[most_deck_bridges_mask]
    
    return most_deck_bridges

# Example

In [ ]:
#cards that we want to find bridges between
deck_monster_names = ['Ash Blossom & Joyous Spring',
                        'Effect Veiler',
                        'Ghost Belle & Haunted Mansion',
                        'Mathmech Addition',
                        'Mathmech Circular',
                        'Mathmech Diameter',
                        'Mathmech Multiplication',
                        'Mathmech Nabla',
                        'Mathmech Sigma',
                        'Mathmech Subtraction',
                        'Parallel eXceed',
                        'Maxx "C"']

#cards that are required to connect with a bridge
required_target_names = ['Mathmech Circular',
                        'Maxx "C"',
                        'Parallel eXceed']

find_best_bridges(deck_monster_names, required_target_names)